In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import datetime
import calendar
import ast

In [1]:
def extract_time_features(row):
    time1, time2 = pd.to_datetime(row.at['start']), pd.to_datetime(row.at['end'])
    #time = pd.to_datetime(row.at['created'])
    #row.at['created'] = pd.to_datetime(row.at['created'])
    if pd.isna(time1) or pd.isna(time2):
        row.at['ds'] = np.nan
        return row
    duration = time2 - time1
    duration_in_s = duration.total_seconds() 
    time_passed = divmod(duration_in_s, 3600)
    ds = f"{time1.year}/{time1.month}/{time1.day}T{time1.hour}:00:00+00:00"
    duration = int(time_passed[0])
    row.at['duration'] = duration
    row.at['ds'] = ds
    return row


In [257]:
fresh_history = fresh_history.apply(extract_time_features, axis=1)

In [8]:
fresh_history = pd.read_csv('fresh_history.csv')

In [143]:
burg_all = pd.read_csv('parkrail-burgdorf.csv', ';')
burg_all['ds'] = 0
burg_all['duration'] = 0
burg_all = burg_all.rename(columns={"Arrival in local time": "start", "Departure in local time": "end"})
burg_all = burg_all.apply(extract_time_features, axis=1)
burg_all

,Parking lot name,xml_id,Arrival in unix time,Departure in unix time,start,end,ds,duration
0,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614038621,1.614053e+09,2021-02-23T01:03:41,2021-02-23T05:11:28,2021/2/23T1:00:00+00:00,4
1,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614059026,1.614065e+09,2021-02-23T06:43:46,2021-02-23T08:17:14,2021/2/23T6:00:00+00:00,1
2,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614078855,1.614079e+09,2021-02-23T12:14:15,2021-02-23T12:16:27,2021/2/23T12:00:00+00:00,0
3,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614084211,1.614085e+09,2021-02-23T13:43:31,2021-02-23T13:49:15,2021/2/23T13:00:00+00:00,0
4,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614145425,1.614146e+09,2021-02-24T06:43:45,2021-02-24T06:49:56,2021/2/24T6:00:00+00:00,0
...,...,...,...,...,...,...,...,...
7139,Burgdorf SBB ParkRail Bucherstrasse 2,17,1613508284,1.613509e+09,2021-02-16T21:44:44,2021-02-16T21:50:37,2021/2/16T21:00:00+00:00,0
7140,Burgdorf SBB ParkRail Bucherstrasse 2,17,1613565844,1.613574e+09,2021-02-17T13:44:04,2021-02-17T16:03:31,2021/2/17T13:00:00+00:00,2
7141,Burgdorf SBB ParkRail Bucherstrasse 2,17,1613601824,1.613602e+09,2021-02-17T23:43:44,2021-02-17T23:49:49,2021/2/17T23:00:00+00:00,0
7142,Burgdorf SBB ParkRail Bucherstrasse 2,17,1613738553,1.613739e+09,2021-02-19T13:42:33,2021-02-19T13:49:53,2021/2/19T13:00:00+00:00,0


In [163]:
facility_names = set(history.facility_name)
facilities_sorted = {}
counter = len(facility_names)
spots = pd.read_csv('parkings.csv').drop(columns=['Unnamed: 0'])
weekend_or_not = lambda x: 0 if x < 5 else 1 
history = fresh_history


for facility in ["Burgdorf"]:
    #try:
        ref = history.loc[history['facility_name'] == facility]
        ref = ref.sort_values(by=['start'])
        max_spots = int(spots.loc[spots['facility'] == str([facility])]['spaces'])
        df = {"ds": [], 'facility': [], 'max_spots': [], 'weekend': [], 'weekday': [], 'hour': []}
        for y in range(2):
            for m in range(1, 13):
                for d in range(1, calendar.monthrange([2020, 2021][y], m)[1]+1):
                    for h in range(24):
                      #print(df)
                      date = f"{[2020, 2021][y]}/{m}/{d}T{h}:00:00+00:00"
                      wd = pd.to_datetime(date).weekday()
                      df['weekday'].append(wd)
                      df['weekend'].append(weekend_or_not(wd))
                      df['facility'].append(facility)
                      df['ds'].append(date)
                      df['max_spots'].append(max_spots)
                      df['hour'].append(h)
        df = pd.DataFrame(df)
        df['taken_digit'] = 0
        
        for ix, entry in ref.iterrows():
            try:
                idx = df.loc[df['ds'] == entry.at['ds']].index[0]
            except:
                continue
            for i in range(int(entry.at['duration'])):
                df.at[idx + i, 'taken_digit'] += 1
        
        burg_all = burg_all.sort_values(by=['start'])
        df['taken_paper'] = 0
        for ix, entry in burg_all.iterrows():
            try:
                idx = df.loc[df['ds'] == entry.at['ds']].index[0]
            except:
                continue
            for i in range(int(entry.at['duration'])):
                df.at[idx + i, 'taken_paper'] += 1
        df['taken_all'] = df['taken_digit'] + df['taken_paper']
        df['perc_free'] = round((df['max_spots'] - df['taken_all']) / df['max_spots'] * 100, 2)
        df.to_csv("tables/" + facility + "_pandemic.csv", index=False)
        counter -= 1
        print(facility, 'saved.', counter, "more to go.")
    #except Exception as e:
    #    print(facility, "failed, because:")
    #    print(e)
                       

Burgdorf saved. 550 more to go.


,start,end,facility_name,ds,duration
0,2020-04-27T07:30:00+00:00,2020-04-28T07:30:00+00:00,Rivera-Bironico,2020/4/27T7:00:00+00:00,24
1,2020-04-27T07:30:00+00:00,2020-04-28T07:30:00+00:00,Fehraltorf,2020/4/27T7:00:00+00:00,24
2,2020-04-27T07:25:00+00:00,2020-04-28T07:25:00+00:00,Bonstetten-Wettswil,2020/4/27T7:00:00+00:00,24
3,2020-04-27T07:25:00+00:00,2020-04-28T07:25:00+00:00,St. Gallen St. Fiden,2020/4/27T7:00:00+00:00,24
4,2020-04-27T07:25:00+00:00,2020-04-28T07:25:00+00:00,St. Gallen St. Fiden,2020/4/27T7:00:00+00:00,24
...,...,...,...,...,...
344384,2020-02-13T06:55:00+00:00,2020-02-14T06:55:00+00:00,Horgen,2020/2/13T6:00:00+00:00,24
344385,2021-02-13T10:25:00+00:00,2021-02-14T10:25:00+00:00,Oberbuchsiten,2021/2/13T10:00:00+00:00,24
344386,2020-02-13T06:55:00+00:00,2020-02-14T06:55:00+00:00,Marthalen,2020/2/13T6:00:00+00:00,24
344387,2021-02-13T10:10:00+00:00,2021-02-17T18:10:00+00:00,Grenchen Süd,2021/2/13T10:00:00+00:00,104


In [58]:
rap = pd.read_csv('park-ride-rapperswil.csv', ';')
rap.Datum = [pd.to_datetime(x) for x in rap.Datum]
rap['hour'] = [x.hour for x in rap.Datum]
rap = rap.sort_values(by='Datum').reset_index(drop=True)
rap_dict = {x:y for x, y in zip(rap.Datum, rap['BELEGUNGSQUOTE (%)'])}

In [229]:
spots = pd.read_csv('parkings.csv').drop(columns=['Unnamed: 0'])
weekend_or_not = lambda x: 0 if x < 5 else 1 
history = fresh_history
facility_names = set(history.facility_name)
facilities_sorted = {}
counter = len(facility_names)

In [61]:
for facility in ["Rapperswil"]:
    #try:
        ref = history.loc[history['facility_name'] == facility]
        ref = ref.sort_values(by=['start'])
        ref.ds = [pd.to_datetime(x) for x in ref.ds]
        max_spots = int(spots.loc[spots['facility'] == str([facility])]['spaces'])
        df = {"ds": [], 'facility': [], 'max_spots': [], 'weekend': [], 'weekday': [], 'hour': []}
        for y in range(2):
            for m in range(1, 13):
                for d in range(1, calendar.monthrange([2020, 2021][y], m)[1]+1):
                    for h in range(24):
                      #print(df)
                      date = f"{[2020, 2021][y]}-{m}-{d}T{h}:00:00+00:00"
                      wd = pd.to_datetime(date).weekday()
                      df['weekday'].append(wd)
                      df['weekend'].append(weekend_or_not(wd))
                      df['facility'].append(facility)
                      df['ds'].append(pd.to_datetime(date))
                      df['max_spots'].append(max_spots)
                      df['hour'].append(h)
                      df['month'].append(m)
                      df['day'].append(d)
        df = pd.DataFrame(df)
        df['taken_digit'] = 0
        
        for ix, entry in ref.iterrows():
            try:
                idx = df.loc[df['ds'] == entry.at['ds']].index[0]
            except:
                continue
            for i in range(int(entry.at['duration'])):
                df.at[idx + i, 'taken_digit'] += 1
        
        df['occupied'] = 0
        for ix in range(3648, len(df['ds'])):
            try:
                df.at[ix, 'occupied'] = rap_dict[df['ds'][ix]]
            except:
                continue
        df['taken_all'] = df['occupied'] * df['max_spots'] / 100
        df['perc_free'] = 100 - df['occupied']
        df.to_csv("tables/" + facility + ".csv", index=False)
        counter -= 1
        print(facility, 'saved.', counter, "more to go.")
    #except Exception as e:
    #    print(facility, "failed, because:")
    #    print(e)
                        

Rapperswil saved. 547 more to go.


In [195]:
passangers = pd.read_csv('passagierfrequenz.csv', ';')
passangers.head()

,Code,Bahnhof_Haltestelle,Bezugsjahr,Kanton,Eigner,DTV,DWV,DNWV,Bemerkungen,Remarques,Note,Remarks,geopos,lod
0,AD,Aadorf,2018,TG,SBB,1700.0,2000.0,1000.0,Durchschnittswert 2018 durch Streckensperrung ...,Valeur moyenne 2018 à la baisse en raison de l...,Valore medio 2018 compromesso dallo sbarrament...,Average value for 2018 impacted by line closure.,"47.4881178542,8.90328450849",http://lod.opentransportdata.swiss/didok/8506013
1,AA,Aarau,2018,AG,SBB,37900.0,44800.0,22700.0,Ohne AVA.,Sans AVA.,Senza AVA.,Without AVA.,"47.3913553369,8.05125354274",http://lod.opentransportdata.swiss/didok/8502113
2,ABO,Aarburg-Oftringen,2018,AG,SBB,2500.0,3000.0,1300.0,NaN,NaN,NaN,NaN,"47.3202667174,7.90820373354",http://lod.opentransportdata.swiss/didok/8502000
3,AAT,Aathal,2018,ZH,SBB,740.0,800.0,610.0,NaN,NaN,NaN,NaN,"47.3359563788,8.76561022548",http://lod.opentransportdata.swiss/didok/8503124
4,ACLA,Acla da Fontauna,2018,GR,MGB,90.0,90.0,80.0,NaN,NaN,NaN,NaN,"46.6971753494,8.84578125896",http://lod.opentransportdata.swiss/didok/8505180


In [235]:
for facility in facility_names:
    try:
        ref = history.loc[history['facility_name'] == facility]
        pas = passangers.loc[passangers['Bahnhof_Haltestelle'] == facility]
        max_spots = int(spots.loc[spots['facility'] == str([facility])]['spaces'])
        try:
            dtv = pas.iloc[0]['DTV']
        except:
            dtv = max_spots * 20
        ref = ref.sort_values(by=['start'])
        ref.ds = [pd.to_datetime(x) for x in ref.ds]
        df = {"ds": [], 'facility': [], 'max_spots': [], 'weekend': [], 'weekday': [], 'hour': [], 'month': [], 'day': [], 'traffic': []}
        for y in range(2):
            for m in range(1, 13):
                for d in range(1, calendar.monthrange([2020, 2021][y], m)[1]+1):
                    for h in range(24):
                      #print(df)
                      date = f"{[2020, 2021][y]}-{m}-{d}T{h}:00:00+00:00"
                      wd = pd.to_datetime(date).weekday()
                      df['weekday'].append(wd)
                      df['weekend'].append(weekend_or_not(wd))
                      df['facility'].append(facility)
                      df['ds'].append(pd.to_datetime(date))
                      df['max_spots'].append(max_spots)
                      df['hour'].append(h)
                      df['month'].append(m)
                      df['day'].append(d)
                      df['traffic'].append(dtv)
        df = pd.DataFrame(df)
        df['taken_digit'] = 0
        for ix, entry in ref.iterrows():
            try:
                idx = df.loc[df['ds'] == entry.at['ds']].index[0]
            except:
                continue
            for i in range(int(entry.at['duration'])):
                df.at[idx + i, 'taken_digit'] += 1
        
        # I assume that digital tickets are on avarage 15% of all the tickets
        df['extrapolated'] = (df['taken_digit'] * 100 / 15).astype('int')

        df['perc_free'] = ((df['max_spots'] - df['extrapolated']) / df['max_spots'] * 100).map(lambda x: 0 if x < 0 else round(x, 2))
        df.to_csv("tables/" + facility + ".csv", index=False)
        counter -= 1
        print(facility, 'saved.', counter, "more to go.")
    except Exception as e:
        print(facility, "failed, because:")
        print(e)
                        

Horgen Oberdorf saved. 534 more to go.
Ossingen saved. 533 more to go.
Moos (BLS) failed, because:
cannot convert the series to <class 'int'>
Schüpfheim saved. 532 more to go.
Kehrsatz (BLS) failed, because:
cannot convert the series to <class 'int'>
Bussigny saved. 531 more to go.
Düdingen saved. 530 more to go.
Busswil saved. 529 more to go.
Auvernier saved. 528 more to go.
Nebikon saved. 527 more to go.
Alpnachstad (zb - Die Zentralbahn) failed, because:
cannot convert the series to <class 'int'>
Rosé saved. 526 more to go.
Räterschen saved. 525 more to go.
Muttenz saved. 524 more to go.
Erlenbach ZH saved. 523 more to go.
Rolle saved. 522 more to go.
Uerikon saved. 521 more to go.
Arnegg saved. 520 more to go.
Dinhard saved. 519 more to go.
Baar saved. 518 more to go.
Hinwil saved. 517 more to go.
Richterswil saved. 516 more to go.
Biglen (BLS) failed, because:
cannot convert the series to <class 'int'>
Gibswil saved. 515 more to go.
Stabio saved. 514 more to go.
Schalunen (RBS) fa

In [ ]:
for facility in facility_names:
    try:
        ref = history.loc[history['facility_name'] == facility]
        pas = passangers.loc[passangers['Bahnhof_Haltestelle'] == facility]
        max_spots = int(spots.loc[spots['facility'] == str([facility])]['spaces'])
        try:
            dtv = pas.iloc[0]['DTV']
        except:
            dtv = max_spots * 20
        ref = ref.sort_values(by=['start'])
        ref.ds = [pd.to_datetime(x) for x in ref.ds]
        df = {"ds": [], 'facility': [], 'max_spots': [], 'weekend': [], 'weekday': [], 'hour': [], 'month': [], 'day': [], 'traffic': []}
        for y in range(2):
            for m in range(1, 13):
                for d in range(1, calendar.monthrange([2020, 2021][y], m)[1]+1):
                    for h in range(24):
                      #print(df)
                      date = f"{[2020, 2021][y]}-{m}-{d}T{h}:00:00+00:00"
                      wd = pd.to_datetime(date).weekday()
                      df['weekday'].append(wd)
                      df['weekend'].append(weekend_or_not(wd))
                      df['facility'].append(facility)
                      df['ds'].append(pd.to_datetime(date))
                      df['max_spots'].append(max_spots)
                      df['hour'].append(h)
                      df['month'].append(m)
                      df['day'].append(d)
                      df['traffic'].append(dtv)
        df = pd.DataFrame(df)
        df['taken_digit'] = 0
        for ix, entry in ref.iterrows():
            try:
                idx = df.loc[df['ds'] == entry.at['ds']].index[0]
            except:
                continue
            for i in range(int(entry.at['duration'])):
                df.at[idx + i, 'taken_digit'] += 1
        
        # I assume that digital tickets are on avarage 15% of all the tickets
        df['extrapolated'] = (df['taken_digit'] * 100 / 15).astype('int')

        df['perc_free'] = ((df['max_spots'] - df['extrapolated']) / df['max_spots'] * 100).map(lambda x: 0 if x < 0 else round(x, 2))
        df.to_csv("tables/" + facility + ".csv", index=False)
        counter -= 1
        print(facility, 'saved.', counter, "more to go.")
    except Exception as e:
        print(facility, "failed, because:")
        print(e)
                        

In [200]:
rapperswil = pd.read_csv('tables/rapperswil_extra.csv', index_col = 'ds')
rapperswil.index = pd.to_datetime(rapperswil.index)
rapperswil['occupied'] = rapperswil['occupied'].replace({0:np.nan})
rapperswil['taken_all'] = rapperswil['taken_all']
rapperswil['taken_paper'] = rapperswil['taken_all'] - rapperswil['taken_digit']
rapperswil.interpolate(method='polynomial', order=2, inplace=True)
rapperswil = rapperswil.dropna()
rapperswil['occupied'] = rapperswil['occupied']
rapperswil.head()

,facility,max_spots,weekend,weekday,hour,taken_digit,occupied,taken_all,perc_free,taken_paper
ds,,,,,,,,,,
2020-06-01 00:00:00+00:00,Rapperswil,180,0,0,0,2,12.0,21.6,88,19.6
2020-06-01 01:00:00+00:00,Rapperswil,180,0,0,1,2,6.0,10.8,94,8.8
2020-06-01 02:00:00+00:00,Rapperswil,180,0,0,2,2,7.0,12.6,93,10.6
2020-06-01 03:00:00+00:00,Rapperswil,180,0,0,3,2,7.0,12.6,93,10.6
2020-06-01 04:00:00+00:00,Rapperswil,180,0,0,4,2,6.0,10.8,94,8.8


In [191]:
rapperswil['perc_paper'] = rapperswil['taken_paper'] / rapperswil['taken_all'] * 100
rapperswil['perc_paper'].where(rapperswil['perc_paper'] >= 0).mean()

81.46560822960286

In [115]:
burg_all['ds'] = 0
burg_all['duration'] = 0
burg_all = burg_all.rename(columns={"Arrival in local time": "start", "Departure in local time": "end"})
burg_all = burg_all.apply(extract_time_features, axis=1)

In [116]:
burg_all.head()

,Parking lot name,xml_id,Arrival in unix time,Departure in unix time,start,end,ds,duration
0,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614038621,1.614053e+09,2021-02-23T01:03:41,2021-02-23T05:11:28,2021/2/23T1:00:00+00:00,4
1,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614059026,1.614065e+09,2021-02-23T06:43:46,2021-02-23T08:17:14,2021/2/23T6:00:00+00:00,1
2,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614078855,1.614079e+09,2021-02-23T12:14:15,2021-02-23T12:16:27,2021/2/23T12:00:00+00:00,0
3,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614084211,1.614085e+09,2021-02-23T13:43:31,2021-02-23T13:49:15,2021/2/23T13:00:00+00:00,0
4,Burgdorf SBB ParkRail Bucherstrasse 2,17,1614145425,1.614146e+09,2021-02-24T06:43:45,2021-02-24T06:49:56,2021/2/24T6:00:00+00:00,0


In [102]:
burg['taken_all'] = df['taken']
burg['paper_tickets'] = df['taken'] - burg['taken_digit']
burg['y'] = burg['taken_all'] / burg['max_spots']

In [103]:
burgdorf = burgdorf[:10081]

In [104]:
burgdorf.to_csv('burgdorf_complete.csv', index=False)

In [169]:
burg = pd.read_csv('tables/Burgdorf_pandemic.csv')
burg = burg.loc[burg['taken_paper'] != 0]
burg['perc_paper'] = burg['taken_paper'] / burg['taken_all'] * 100
burg['perc_paper'].mean()

93.41398673570765

In [2]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier, GradientBoostingClassifier
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [115]:
rapperswil.columns

Index(['facility', 'max_spots', 'weekend', 'weekday', 'hour', 'taken_digit',
       'occupied', 'taken_all', 'perc_free'],
      dtype='object')

In [202]:
X = rapperswil.drop(columns=['facility', 'taken_digit', 'taken_paper', 'taken_all', 'perc_free', 'occupied'])
print(X.columns)
y = rapperswil['perc_free']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.6, random_state=13, shuffle = False)

params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.1,
          'loss': 'ls'}

cv = KFold(n_splits=10, shuffle=False)

reg_rap = GBR(**params)
reg_rap.fit(X_train, y_train)
scores = cross_val_score(reg_rap, X, y, scoring='r2', cv=cv, n_jobs=-1)

mse = mean_squared_error(y_test, reg_rap.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print(scores)

Index(['max_spots', 'weekend', 'weekday', 'hour'], dtype='object')
The mean squared error (MSE) on test set: 392.3684
[0.82535236 0.74403571 0.66297019 0.78775792 0.84182682 0.84914235
 0.84577427 0.58479078 0.5549593  0.80109315]


In [160]:
X = burg.drop(columns=['ds', 'facility', 'taken_digit', 'taken_paper', 'taken_all', 'perc_free', 'extrapolated'])
print(X)
y = burg['perc_free']
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.6, random_state=13, shuffle = True)

params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

cv = KFold(n_splits=10, random_state=1, shuffle=True)

reg = GBR(**params)
reg.fit(X_train, y_train)
scores = cross_val_score(reg, X, y, scoring='r2', cv=cv, n_jobs=-1)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))
print(scores)

       max_spots  weekend  weekday
8784         155        0        4
8785         155        0        4
8786         155        0        4
8787         155        0        4
8788         155        0        4
...          ...      ...      ...
10193        155        1        6
10194        155        1        6
10195        155        1        6
10196        155        1        6
10197        155        1        6

[1412 rows x 3 columns]
The mean squared error (MSE) on test set: 173.5107
[0.20209234 0.18102343 0.10632283 0.10560712 0.07082202 0.16911468
 0.17791733 0.15431806 0.1858364  0.19626025]


In [249]:
import os
from sklearn.linear_model import SGDRegressor

reg = SGDRegressor()
for facility in os.listdir('tables/'):
    df = pd.read_csv('tables/' + facility)
    X = df.drop(columns = ['ds', 'facility', 'taken_digit', 'perc_free', 'extrapolated'])
    y = df['perc_free']
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13, shuffle = False)                       

    model.fit(X_train, y_train)
    mse = mean_squared_error(y_test, model.predict(X_test))
    print("The mean squared error (MSE) on test set: {:.4f}".format(mse))


384/384 [==============================] - 1s 1ms/step - loss: 10798.1824
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 9367.6797
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 5958.5259
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 1s 2ms/step - loss: 6869.5317
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 7898.9844
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 6067.9429
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 8982.9307
The mean squared error (MSE) on test set: 10000.0000
384/384 [==============================] - 0s 1ms/step - loss: 8758.4824
The mean squared error (MSE) on test set: 10

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [7]:
import os
files = os.listdir('tables/')
merged = pd.read_csv('tables/' + files[0])
for facility in files[1:]:
    if facility[0] != '.':
        merged = pd.concat([merged, pd.read_csv('tables/' + facility)])
    

In [26]:
merged.columns

Index(['ds', 'facility', 'max_spots', 'weekend', 'weekday', 'hour', 'month',
       'day', 'traffic', 'taken_digit', 'extrapolated', 'perc_free',
       'facility_id', 'probability'],
      dtype='object')

In [22]:
from sklearn.preprocessing import OrdinalEncoder
def classify_perc(x):
    if x > 66:
        return 2
    if x < 33:
        return 0
    else:
        return 1
    
enc = OrdinalEncoder()
merged['facility_id'] = merged['facility']
merged['facility_id'] = enc.fit_transform(np.array(merged['facility_id']).reshape(-1,1))
merged['probability'] = merged['perc_free'].map(classify_perc)

In [8]:
data = pd.read_csv('complete_sbb_data.csv').dropna()

In [9]:
X = data.drop(columns = ['ds', 'facility', 'taken_digit', 'perc_free', 'extrapolated', 'probability'])
y = data['probability']
cv = KFold(n_splits=5, random_state=1, shuffle=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=13, shuffle = False) 

classificator = GradientBoostingClassifier()
classificator = classificator.fit(X_train, y_train)
sc = classificator.score(X_test, y_test)
print(sc)
#scores = cross_val_score(classificator, X, y, cv=cv, n_jobs=-1)


0.8908437929267614


In [10]:
import sklearn, pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(classificator, file)

In [11]:
rfc = RandomForestClassifier()
rfc = rfc.fit(X_train, y_train)
sc = rfc.score(X_test, y_test)

In [12]:
print(sc)

0.8850467493771628


In [18]:
model.fit(merged)
X = merged.drop(columns = ['ds', 'facility', 'taken_digit', 'perc_free', 'extrapolated'])
y = merged['perc_free']

cv = KFold(n_splits=10, random_state=1, shuffle=True)

scores = cross_val_score(model, X, y, scoring='r2', cv=cv, n_jobs=-1)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).

In [256]:
import sklearn, pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(reg, file)

In [205]:
!pip install tensorflow

     |██████████████████████████▎     | 323.5 MB 96.2 MB/s eta 0:00:012

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 394.3 MB 8.7 kB/s  eta 0:00:01
     |████████████████████████████████| 129 kB 79.1 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 29.0 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 2.9 MB 79.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 4.9 MB/s  eta 0:00:01
     |████████████████████████████████| 42 kB 1.6 MB/s  eta 0:00:01
     |████████████████████████████████| 462 kB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 10.6 MB 81.6 MB/s eta 0:00:01
     |████████████████████████████████| 298 kB 81.6 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 69.8 MB/s eta 0:00:01
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4829 sha256=7a3d6efdec9a535c6643d7eb53ea565cfbb4c6136e6466316252013d9b64b585
  Stored in directory: /home/jupyter/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb3

In [22]:
import tensorflow
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import activations
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [43]:
def baseline_model():
    input_layer = Input(shape=(8), dtype='float32')
    dense1 = Dense(64, activation=activations.swish)(input_layer)
    dense2 = Dense(128, activation=activations.swish)(dense1)
    dense3 = Dense(64, activation=activations.sigmoid)(dense2)
    dropout_layer = Dropout(0.3)(dense3)
    output_layer = Dense(3, activation=activations.sigmoid)(dropout_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='categorical_crossentropy', metrics=["accuracy"], optimizer='SGD')
    return model

In [44]:

model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 7)]               0         
_________________________________________________________________
dense_3 (Dense)              (None, 60)                480       
_________________________________________________________________
dense_4 (Dense)              (None, 60)                3660      
_________________________________________________________________
dropout_1 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 183       
Total params: 4,323
Trainable params: 4,323
Non-trainable params: 0
_________________________________________________________________


In [ ]:
estimator = KerasClassifier(build_fn = baseline_model, epochs = 7, batch_size = 64, verbose = 0)
kfold = KFold(n_splits = 5, shuffle = True, random_state = 0)
# Object to describe the result
results = cross_val_score(estimator, X, y, cv = kfold)
# Result
print("Result: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [34]:
model.save('model.json')

INFO:tensorflow:Assets written to: model.json/assets


In [42]:
!zip -r model.zip "model.json"

  adding: model.json/ (stored 0%)
  adding: model.json/saved_model.pb (deflated 88%)
  adding: model.json/assets/ (stored 0%)
  adding: model.json/variables/ (stored 0%)
  adding: model.json/variables/variables.data-00000-of-00001 (deflated 18%)
  adding: model.json/variables/variables.index (deflated 48%)


In [26]:
X = data.drop(columns = ['ds', 'facility', 'taken_digit', 'perc_free', 'extrapolated', 'probability'])
y = data['probability']



/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 

Result: nan% (nan%)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:614: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 593, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 223, in fit
    return super(KerasClassifier, self).fit(x, y, **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/wrappers/scikit_learn.py", line 166, in fit
    history = self.model.fit(x, y, **fit_args)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py", line 1100, in fit
    tmp_logs = self.train_function(iterator)
  File "/opt/conda/lib/python3.7/site-packages/tensorflow/python/eager/def_function.py", line 